### Importing the necessary libraries and packages

In [ ]:
from sklearn.model_selection import train_test_split # splitting the dataset
from sklearn.preprocessing import StandardScaler # data preprocessing
from sklearn.tree import DecisionTreeClassifier # classification algorithm
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, classification_report, roc_auc_score # metrics
import pandas as pd # data processing
import matplotlib.pyplot as plt # data plotting

### Importing the csv file as a dataFrame

In [ ]:
df=pd.read_csv("Heart.csv")
df.head()

### Now taking 2 parts from the dataset:
#### X : which contains all the data except the <b>target</b> column<br>y : which contains only the <b>target</b> column 

In [ ]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
print("//Independent features//")
print(X.head())
print("\n\n//Dependent feature//")
print(y.head())

### Split the X and y dataframes for test and training

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=42)

### Scaling the dataset

In [ ]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# Decision Tree Classification

In [ ]:
# Decision Tree Classification
dt = DecisionTreeClassifier(criterion = 'entropy',random_state=42,max_depth=6)
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)
dt_con_matrix = confusion_matrix(y_test, dt_pred)
dt_acc = accuracy_score(y_test, dt_pred)
print("Confusion Matrix",'\n',dt_con_matrix,"\n")
print("Accuracy of Decision Tree Classification : ",dt_acc*100,"\n")
print("Classification Report",'\n',classification_report(y_test,dt_pred))

In [ ]:
y_score = dt.predict_proba(X_test)[:,1]
false_positive_rate, true_positive_rate, threshold = roc_curve(y_test, y_score)

### Plotting the ROC curve

In [ ]:
plt.figure(figsize=(5,5),dpi=150)
plt.plot(false_positive_rate,true_positive_rate, color='red', label = "Decision Tree Classification")
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],linestyle='--')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

### Computing the AUC of ROC

In [ ]:
print("Area Under the Receiver Operating Characteristic Curve :",roc_auc_score(y_test,y_score))